In [1]:
import fitz
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import sys
import pandas as pd

In [2]:
# !pip install PyMuPDF

In [3]:
pdf_path = "data\\Datos_abiertos_historicos_etv_2022.pdf"

In [4]:
def extract_links(pdf_path):
    links = []
    doc = fitz.open(pdf_path)    
    page = doc[0]
    for link in page.get_links():
        links.append(
            link['uri']
        )
    doc.close()
    return links

In [5]:
def descompresor(links):
    carpeta = 'data\csv-files'
    success = 0
    fail_download = 0
    fail_process = 0
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    ruta_bitacora = 'data\\bitacora.txt'
    with open(ruta_bitacora, 'w') as log_file:

        #Redirigir la salida estándar al archivo
        sys.stdout = log_file
        for link in links:
            try:
                respuesta = requests.get(link)
                if respuesta.status_code == 200:
                    with ZipFile(BytesIO(respuesta.content)) as archivo_zip:
                        archivo_zip.extractall(carpeta)
                        print(f'Archivo descomprimido desde {link} en {carpeta}')
                        success+=1
                else:
                    print(f'Error al descargar {link}. Código de estado: {respuesta.status_code}')
                    fail_download+=1

            except Exception as e:
                print(f'Error al procesar {link}: {str(e)}')
                fail_process+=1
        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")
        
        #Restaurar la salida estándar al archivo
        sys.stdout = sys.__stdout__

        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")

    print(f'Los resultados se han guardado en {ruta_bitacora}')

In [6]:
enlaces = extract_links(pdf_path)

descompresor(enlaces)

In [ ]:
cvs_files_path = 'data\csv-files'

39

In [ ]:
def get_dataframe(carpeta):
    dataframe = pd.DataFrame()
    # Recorrer todos los archivos en la carpeta
    for archivo_csv in os.listdir(carpeta):
        if archivo_csv.endswith('.csv'):
            ruta_completa = os.path.join(carpeta, archivo_csv)
            df = pd.read_csv(ruta_completa)
            dataframe = pd.concat([dataframe, df])
    return dataframe

In [ ]:
dengue_2022 = get_dataframe(cvs_files_path)

In [ ]:
dengue_2022

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2022-05-30,934897,2,23,12,29,2,2,12,29,...,2,2,2,2,5.0,2,5,1,12,29
1,2022-05-30,934961,1,12,30,204,2,2,30,204,...,2,2,2,2,5.0,1,5,1,30,204
2,2022-05-30,934968,2,42,14,39,2,2,14,39,...,2,2,2,2,5.0,1,5,1,14,39
3,2022-05-30,934974,1,35,30,118,2,2,30,44,...,2,2,2,2,5.0,1,5,3,30,118
4,2022-05-30,934916,1,49,30,44,2,2,30,44,...,2,2,2,2,5.0,2,5,1,30,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42758,2022-10-24,976782,1,37,26,42,2,2,26,42,...,2,2,2,2,5.0,2,5,1,26,42
42759,2022-10-24,976783,1,25,26,42,2,2,26,42,...,2,2,2,2,5.0,1,5,1,26,42
42760,2022-10-24,976786,1,26,26,18,2,2,26,18,...,2,2,2,2,5.0,1,5,1,26,18
42761,2022-10-24,976787,1,1,26,42,2,2,26,42,...,2,2,2,2,5.0,1,5,1,26,42


In [ ]:
dengue_2022['FECHA_ACTUALIZACION'].unique() #Se indentifican formatos

array(['2022-05-30', '2022-01-31', '2022-02-28', '2022-05-02',
       '2022-01-06', '2022-04-04', '2022-07-04', '2022-06-06',
       '2022-02-08', '2022-03-07', '2022-05-09', '2022-01-10',
       '2022-04-11', '2022-07-11', '2022-06-13', '2022-03-14',
       '2022-02-14', '2022-05-16', '2022-01-17', '2022-04-18',
       '2022-07-18', '25/07/2022', '2022-08-01', '2022-08-09',
       '2022-08-15', '2022-08-22', '2022-08-29', '2022-09-05',
       '2022-09-13', '2022-09-21', '2022-06-20', '2022-03-22',
       '2022-05-23', '2022-01-24', '25/04/2022', '2022-09-26',
       '2022-06-27', '2022-03-28', '2022-10-24'], dtype=object)

In [ ]:
#Se cambian a objetos datetime los formatos mixtos
date1 = pd.to_datetime(dengue_2022['FECHA_ACTUALIZACION'], errors='coerce', format='%Y-%m-%d')
date2 = pd.to_datetime(dengue_2022['FECHA_ACTUALIZACION'], errors='coerce', format='%d/%m/%Y')
dengue_2022['FECHA_ACTUALIZACION'] = date1.fillna(date2)
#Se organizan las fechas
dengue_2022 = dengue_2022.sort_values(by='FECHA_ACTUALIZACION')

In [ ]:
dengue_2022

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
31430,2022-01-06,923165,1,27,24,32,2,2,24,32,...,2,2,2,2,5.0,2,5,1,24,32
16725,2022-01-06,891367,2,10,18,8,2,2,18,8,...,2,2,2,2,5.0,2,5,1,18,8
16726,2022-01-06,890365,1,24,30,193,2,2,30,193,...,2,2,2,2,5.0,1,5,3,30,193
16727,2022-01-06,890368,1,25,16,52,2,2,16,52,...,2,2,2,2,5.0,2,5,1,16,52
16728,2022-01-06,890371,2,31,16,6,2,2,16,6,...,2,2,2,2,5.0,2,5,1,16,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14255,2022-10-24,975728,2,12,6,7,2,2,6,7,...,2,2,2,2,5.0,1,5,2,6,7
14256,2022-10-24,975730,2,62,12,66,1,1,12,66,...,2,2,2,2,5.0,1,5,2,12,66
14257,2022-10-24,975731,2,53,31,41,2,2,31,41,...,2,2,2,2,5.0,2,5,1,31,41
14259,2022-10-24,975722,2,56,26,18,2,2,26,18,...,2,2,2,2,5.0,1,5,1,26,18
